# Version 02 - Metric/Feature extraction and calculation 

In [8]:
import numpy as np
import librosa
import soundfile as sf
import webrtcvad
import pyloudnorm as pyln
from faster_whisper import WhisperModel
from typing import Dict, Any, Optional, List

In [ ]:
# ============================================================
# FEATURE EXTRACTION (Enhanced with Priority Metrics)
# ============================================================

AUDIO_PATH = "/Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_slice_20251203_170320.mp3"

# 1) Load audio (mono, 16 kHz)
y, sr = sf.read(AUDIO_PATH)
if y.ndim > 1:
    y = np.mean(y, axis=1)
target_sr = 16000
if sr != target_sr:
    y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
    sr = target_sr

duration_sec = len(y) / sr

# 2) VAD-based pause ratio with webrtcvad
vad = webrtcvad.Vad(2)  # 0–3, higher = more aggressive
frame_ms = 30  #webrtcvad library only accepts frames of exactly 10, 20, or 30 milliseconds
frame_len = int(sr * frame_ms / 1000)
num_frames = len(y) // frame_len
speech_frames = 0

# webrtcvad expects 16-bit PCM bytes
pcm = (y * 32767).astype(np.int16).tobytes()
for i in range(num_frames):
    start = i * frame_len * 2  # 2 bytes per sample
    end = start + frame_len * 2
    frame = pcm[start:end]
    if len(frame) < frame_len * 2:
        break
    if vad.is_speech(frame, sr):
        speech_frames += 1

speech_time = speech_frames * frame_ms / 1000.0
pause_time = duration_sec - speech_time
pause_ratio = max(pause_time, 0.0) / max(duration_sec, 1e-6)

# 3) Pitch variation via librosa.pyin (F0 in Hz)
f0, voiced_flag, _ = librosa.pyin(
    y,
    fmin=librosa.note_to_hz("C2"),
    fmax=librosa.note_to_hz("C7"),
    sr=sr
)
f0_voiced = f0[~np.isnan(f0)]
if len(f0_voiced) > 0:
    pitch_mean = float(np.mean(f0_voiced))
    pitch_std = float(np.std(f0_voiced))
    pitch_range = float(np.max(f0_voiced) - np.min(f0_voiced))

    # ✨ NEW: Pitch RMS Deviation (key prosody/charisma metric)
    # RMS deviation of F0 from mean = sqrt(mean((f0 - mean)^2))
    # This is equivalent to std, but we calculate it explicitly for clarity
    pitch_rms_deviation = float(np.sqrt(np.mean((f0_voiced - pitch_mean) ** 2)))
else:
    pitch_mean = pitch_std = pitch_range = pitch_rms_deviation = np.nan

# 4) Volume stability via RMS
frame_len_rms = int(0.05 * sr)  # 50ms frames
hop_len_rms = frame_len_rms // 2
rms = librosa.feature.rms(y=y, frame_length=frame_len_rms, hop_length=hop_len_rms)[0]
rms_mean = float(np.mean(rms))
rms_std = float(np.std(rms))
rms_cv = float(rms_std / (rms_mean + 1e-8))

# ✨ NEW: Short-term Loudness Variance (direct emotionality indicator)
# Variance of RMS values across frames
loudness_variance = float(np.var(rms))

# ✨ NEW: Crest Factor (dynamic range / vocal punch)
# Crest Factor (dB) = 20 * log10(peak / RMS)
# Higher = more dynamic emphasis and articulation bursts
peak_amplitude = float(np.max(np.abs(y)))
overall_rms = float(np.sqrt(np.mean(y ** 2)))
if overall_rms > 1e-8:
    crest_factor_db = float(20 * np.log10(peak_amplitude / overall_rms))
else:
    crest_factor_db = 0.0

# 5) Integrated loudness (LUFS) - for normalization reference only
meter = pyln.Meter(sr)
lufs = float(meter.integrated_loudness(y))

# 6) Speech rate via faster-whisper (WPM)
model = WhisperModel("small", device="cpu", compute_type="int8")
segments, _ = model.transcribe(AUDIO_PATH, beam_size=1)
words = 0
first_t = None
last_t = None
for seg in segments:
    text = seg.text.strip()
    if not text:
        continue
    seg_words = text.split()
    words += len(seg_words)
    if first_t is None:
        first_t = seg.start
    last_t = seg.end

if words > 0 and last_t is not None and first_t is not None:
    spoken_dur_min = (last_t - first_t) / 60.0
    wpm = words / max(spoken_dur_min, 1e-6)
else:
    wpm = 0.0

# 7) Acoustic speech rate proxy: voiced frames per second from pyin
voiced_rate = float(np.mean(voiced_flag)) * (len(voiced_flag) / duration_sec) if duration_sec > 0 else 0.0

print("\n=== PRIORITY CAPTIVATION METRICS ===")
print(f"Pause Ratio: {pause_ratio:.3f} (fraction of non-speech time)")
print(f"Pitch Std: {pitch_std:.2f} Hz (expressiveness)")
print(f"Pitch Range: {pitch_range:.2f} Hz (vocal dynamism)")
print(f"Pitch RMS Deviation: {pitch_rms_deviation:.2f} Hz (prosodic melody)")
print(f"Loudness Variance: {loudness_variance:.6f} (vocal emotionality)")
print(f"Crest Factor: {crest_factor_db:.2f} dB (vocal punch/emphasis)")
print(f"Speech Rate (WPM): {wpm:.1f}")
print(f"Voiced Frames/sec: {voiced_rate:.2f} (fluency proxy)")

print("\n=== SUPPORTIVE METRICS ===")
print(f"Pitch Mean: {pitch_mean:.2f} Hz (reference baseline)")
print(f"RMS Mean/Std/CV: {rms_mean:.4f} / {rms_std:.4f} / {rms_cv:.3f}")

print("\n=== NORMALIZATION REFERENCE ONLY ===")
print(f"Integrated Loudness (LUFS): {lufs:.1f} (not for scoring engagement)")

# ============================================================
# INTERPRETATION FUNCTIONS (Updated Priority)
# ============================================================

def interpret_pause_ratio(pause_ratio: float) -> Dict[str, Any]:
    """Interpret pause ratio - HIGH PRIORITY for captivation."""
    if pause_ratio < 0.05:
        label = "very_low_pauses"
        interpretation = "Almost no silence; speech likely feels rushed or dense."
        coaching = "Add short pauses (0.5-1 second) after key ideas for comprehension."
    elif pause_ratio < 0.15:
        label = "low_pauses"
        interpretation = "Some pauses, but still quite continuous and fast-paced."
        coaching = "Use slightly longer pauses after important points for emphasis."
    elif pause_ratio < 0.35:
        label = "balanced_pauses"
        interpretation = "Healthy balance; rhythm likely feels natural and engaging."
        coaching = "Excellent pause usage; maintain this natural rhythm."
    elif pause_ratio < 0.50:
        label = "many_pauses"
        interpretation = "Many or long pauses; aids clarity but may sound hesitant."
        coaching = "Connect sentences more smoothly while keeping key strategic pauses."
    else:
        label = "very_high_pauses"
        interpretation = "Over 50% silence; speech may feel fragmented or disjointed."
        coaching = "Reduce long pauses and maintain more continuous flow between ideas."

    return {
        "value": pause_ratio,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_pitch_expressiveness(
    pitch_mean: float,
    pitch_std: float,
    pitch_range: float,
    pitch_rms_deviation: float,
    speaker_profile: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Interpret pitch-based expressiveness - HIGH PRIORITY for captivation.
    Focuses on Std, Range, and RMS Deviation as key charisma indicators.
    """

    # Pitch Std (High Priority)
    if pitch_std < 15:
        std_label = "very_flat"
        std_interp = "Minimal pitch movement; sounds monotone or robotic."
        std_coaching = "Add pitch rises on important words and falls at sentence ends."
    elif pitch_std < 25:
        std_label = "low_variation"
        std_interp = "Limited pitch movement; functional but not expressive."
        std_coaching = "Increase pitch contrast on key ideas to sound more dynamic."
    elif pitch_std < 40:
        std_label = "moderate"
        std_interp = "Some pitch movement; approaching engaging levels."
        std_coaching = "Good baseline; push slightly more variation on emotional content."
    elif pitch_std < 60:
        std_label = "expressive"
        std_interp = "Strong pitch movement; natural and highly engaging."
        std_coaching = "Excellent expressiveness; maintain this dynamic quality."
    else:
        std_label = "very_wide"
        std_interp = "Very large pitch movement; theatrical and highly emotional."
        std_coaching = "Consider smoothing extremes unless intentionally dramatic."

    # Pitch Range (High Priority)
    if pitch_range < 50:
        range_label = "very_narrow"
        range_interp = "Extremely limited pitch span; reinforces monotone."
        range_coaching = "Practice using wider pitch on questions and emphasis words."
    elif pitch_range < 100:
        range_label = "narrow"
        range_interp = "Limited pitch span; reduces vocal dynamism."
        range_coaching = "Expand your pitch range on key phrases for more energy."
    elif pitch_range < 200:
        range_label = "normal"
        range_interp = "Typical pitch span for conversational speech."
        range_coaching = "Good range; focus on strategic placement of pitch changes."
    elif pitch_range < 300:
        range_label = "wide"
        range_interp = "Large pitch span; indicates expressive delivery."
        range_coaching = "Excellent vocal dynamism; ensure it matches your message."
    else:
        range_label = "very_wide"
        range_interp = "Very large pitch span; highly theatrical or emotional."
        range_coaching = "Maintain this range only if appropriate for your audience and context."

    # Pitch RMS Deviation (High Priority - "vocal melody")
    if pitch_rms_deviation < 15:
        rms_label = "flat_prosody"
        rms_interp = "Low melodic variation; lacks charismatic vocal melody."
        rms_coaching = "Work on creating more 'musical' pitch patterns in your speech."
    elif pitch_rms_deviation < 25:
        rms_label = "moderate_prosody"
        rms_interp = "Some melodic quality; could be more captivating."
        rms_coaching = "Enhance prosodic melody by varying pitch more on important words."
    elif pitch_rms_deviation < 45:
        rms_label = "melodic"
        rms_interp = "Good vocal melody; engaging prosody that holds attention."
        rms_coaching = "Strong prosodic expressiveness; this enhances your charisma."
    else:
        rms_label = "highly_melodic"
        rms_interp = "Very high vocal melody; highly captivating prosody."
        rms_coaching = "Exceptional melodic expressiveness; maintain for high-impact moments."

    # Mean context (reference only, not for scoring)
    mean_context = None
    if speaker_profile == "male":
        if pitch_mean < 80:
            mean_context = "Mean pitch very low for typical adult male."
        elif pitch_mean > 155:
            mean_context = "Mean pitch relatively high for typical adult male."
        else:
            mean_context = "Mean pitch within typical adult male range."
    elif speaker_profile == "female":
        if pitch_mean < 165:
            mean_context = "Mean pitch relatively low for typical adult female."
        elif pitch_mean > 255:
            mean_context = "Mean pitch relatively high for typical adult female."
        else:
            mean_context = "Mean pitch within typical adult female range."

    coaching = f"Variation: {std_coaching} Range: {range_coaching} Prosody: {rms_coaching}"

    return {
        "values": {
            "mean_hz": pitch_mean,
            "std_hz": pitch_std,
            "range_hz": pitch_range,
            "rms_deviation_hz": pitch_rms_deviation,
        },
        "labels": {
            "std": std_label,
            "range": range_label,
            "rms_deviation": rms_label,
        },
        "interpretation": {
            "std": std_interp,
            "range": range_interp,
            "rms_deviation": rms_interp,
            "mean_context": mean_context,
        },
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_loudness_variance(loudness_variance: float) -> Dict[str, Any]:
    """Interpret short-term loudness variance - HIGH PRIORITY for emotionality."""

    # These thresholds should be calibrated on your data
    if loudness_variance < 0.0001:
        label = "very_flat"
        interpretation = "Extremely consistent loudness; lacks emotional texture."
        coaching = "Add dynamic loudness changes to emphasize important moments."
    elif loudness_variance < 0.0005:
        label = "low_variation"
        interpretation = "Low loudness variation; sounds controlled but flat."
        coaching = "Increase volume on key words and soften transitions for more emotion."
    elif loudness_variance < 0.002:
        label = "moderate"
        interpretation = "Moderate loudness variation; natural vocal dynamics."
        coaching = "Good emotional texture; use louder emphasis strategically."
    elif loudness_variance < 0.005:
        label = "high_variation"
        interpretation = "High loudness variation; emotionally expressive delivery."
        coaching = "Excellent dynamic range; ensure quieter parts remain audible."
    else:
        label = "very_high"
        interpretation = "Very high loudness swings; highly emotional or inconsistent."
        coaching = "Maintain steadier baseline while preserving intentional emphasis."

    return {
        "value": loudness_variance,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_crest_factor(crest_factor_db: float) -> Dict[str, Any]:
    """
    Interpret crest factor - HIGH PRIORITY for vocal punch/excitement.
    Typical speech crest factor: ~12 dB [web:27][web:30]
    """

    if crest_factor_db < 8:
        label = "very_compressed"
        interpretation = "Very low dynamic range; sounds flat and overly compressed."
        coaching = "Add more articulation emphasis with sharper consonants and stressed syllables."
    elif crest_factor_db < 11:
        label = "low_dynamics"
        interpretation = "Below typical speech dynamics; lacks punch and excitement."
        coaching = "Increase contrast between emphasized and normal words for more impact."
    elif crest_factor_db < 14:
        label = "normal"
        interpretation = "Typical speech dynamics; natural balance of peaks and average level."
        coaching = "Good dynamic balance; maintain clear articulation on key points."
    elif crest_factor_db < 18:
        label = "high_dynamics"
        interpretation = "High dynamic range; energetic with strong articulation bursts."
        coaching = "Excellent vocal punch; ensure average level remains comfortable."
    else:
        label = "very_high"
        interpretation = "Very high dynamic range; extremely punchy or inconsistent."
        coaching = "Consider smoothing extreme peaks while maintaining overall energy."

    return {
        "value": crest_factor_db,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_speech_rate_wpm(wpm: float) -> Dict[str, Any]:
    """Interpret speech rate - HIGH PRIORITY for engagement."""

    if wpm < 100:
        label = "very_slow"
        interpretation = "Very slow pace; can feel heavy or overly deliberate."
        coaching = "Increase tempo on less critical details to add energy."
    elif wpm < 130:
        label = "slow"
        interpretation = "Slower than typical; very clear but may lack energy."
        coaching = "Speed up slightly while maintaining clarity on important points."
    elif wpm < 160:
        label = "optimal"
        interpretation = "Optimal range for presentations; easy to follow and engaging."
        coaching = "Excellent pace; maintain this speed and use pauses for emphasis."
    elif wpm < 190:
        label = "fast"
        interpretation = "Fast pace; energetic but can reduce comprehension."
        coaching = "Slow down slightly and add clearer pauses between key points."
    else:
        label = "very_fast"
        interpretation = "Very fast; listeners may miss information or feel overwhelmed."
        coaching = "Deliberately slow down and pause more so important points land."

    return {
        "value": wpm,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_acoustic_speech_rate(voiced_rate: float) -> Dict[str, Any]:
    """Interpret acoustic speech rate proxy - HIGH PRIORITY for fluency."""

    if voiced_rate < 15:
        label = "sparse_voicing"
        interpretation = "Very low voicing density; suggests slow speech or many pauses."
        coaching = "Reduce pause lengths slightly or increase speaking tempo."
    elif voiced_rate < 22:
        label = "below_average"
        interpretation = "Lower voicing density; deliberate pacing or pause-heavy."
        coaching = "Ensure pauses are strategic, not hesitation-based."
    elif voiced_rate < 30:
        label = "normal"
        interpretation = "Typical voicing density for moderate-paced speech."
        coaching = "Good fluency baseline; combine with clear pauses and articulation."
    elif voiced_rate < 38:
        label = "high_density"
        interpretation = "Dense voicing; suggests fast speech or minimal pauses."
        coaching = "Insert brief pauses after key ideas to avoid sounding rushed."
    else:
        label = "very_high"
        interpretation = "Very dense voicing; rapid speech with very little silence."
        coaching = "Add breathing room with short pauses so listeners can process."

    return {
        "value": voiced_rate,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_volume_stability(rms_mean: float, rms_std: float, rms_cv: float) -> Dict[str, Any]:
    """Interpret volume stability - SUPPORTIVE metric (not primary)."""

    if rms_cv < 0.30:
        label = "very_stable"
        interpretation = "Very consistent loudness; highly controlled."
        coaching = "If too flat, add small dynamic emphasis on key sentences."
    elif rms_cv < 0.60:
        label = "stable"
        interpretation = "Moderate volume changes; sounds natural."
        coaching = "Good stability; ensure quieter words remain audible."
    elif rms_cv < 1.00:
        label = "variable"
        interpretation = "Noticeable volume swings; some inconsistency."
        coaching = "Maintain steadier microphone distance and avoid trailing off."
    else:
        label = "highly_variable"
        interpretation = "Strong volume swings; can be distracting."
        coaching = "Aim for more consistent baseline; reserve big changes for emphasis."

    return {
        "values": {
            "rms_mean": rms_mean,
            "rms_std": rms_std,
            "rms_cv": rms_cv,
        },
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "SUPPORTIVE"
    }

def interpret_loudness_lufs(lufs: float) -> Dict[str, Any]:
    """
    Interpret LUFS - NORMALIZATION REFERENCE ONLY (not for scoring engagement).
    Recording-dependent; influenced by microphone distance and equipment.
    """

    if lufs < -26:
        label = "very_quiet"
        note = "Recording level is low; this is a technical issue, not speech quality."
        coaching = "Increase recording gain or move closer to microphone (aim for -20 to -18 LUFS)."
    elif lufs < -22:
        label = "quiet"
        note = "Recording below comfortable range; technical adjustment needed."
        coaching = "Raise recording level slightly for better playback comfort."
    elif lufs < -18:
        label = "comfortable"
        note = "Recording level is in good range; no technical adjustments needed."
        coaching = "Loudness normalization is appropriate; focus on performance metrics."
    elif lufs < -14:
        label = "loud"
        note = "Recording louder than typical speech; may cause listener fatigue."
        coaching = "Lower recording level slightly to reduce fatigue risk."
    else:
        label = "very_loud"
        note = "Recording is very loud; risk of distortion and listener fatigue."
        coaching = "Reduce recording gain or move further from microphone."

    return {
        "value": lufs,
        "label": label,
        "interpretation": note,
        "coaching": coaching,
        "priority": "NORMALIZATION_ONLY",
        "note": "⚠️ Use only for technical normalization, NOT for speech performance scoring."
    }

# ============================================================
# COMPREHENSIVE SPEECH EVALUATION
# ============================================================

def evaluate_speech_performance(
    pause_ratio: float,
    pitch_mean: float,
    pitch_std: float,
    pitch_range: float,
    pitch_rms_deviation: float,
    loudness_variance: float,
    crest_factor_db: float,
    rms_mean: float,
    rms_std: float,
    rms_cv: float,
    lufs: float,
    wpm: float,
    voiced_rate: float,
    speaker_profile: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Comprehensive speech evaluation focused on captivation metrics.
    Returns prioritized analysis with coaching recommendations.
    """

    # Interpret all features
    pause_info = interpret_pause_ratio(pause_ratio)
    pitch_info = interpret_pitch_expressiveness(
        pitch_mean, pitch_std, pitch_range, pitch_rms_deviation, speaker_profile
    )
    loudness_var_info = interpret_loudness_variance(loudness_variance)
    crest_info = interpret_crest_factor(crest_factor_db)
    wpm_info = interpret_speech_rate_wpm(wpm)
    voiced_info = interpret_acoustic_speech_rate(voiced_rate)

    # Supportive metrics
    volume_info = interpret_volume_stability(rms_mean, rms_std, rms_cv)
    lufs_info = interpret_loudness_lufs(lufs)

    # Collect high-priority coaching
    priority_coaching = []
    for item in [pause_info, pitch_info, loudness_var_info, crest_info, wpm_info, voiced_info]:
        if item.get("priority") == "HIGH":
            priority_coaching.append(item.get("coaching"))

    # Supportive coaching
    supportive_coaching = [volume_info.get("coaching")]

    # Technical note (not coaching)
    technical_note = lufs_info.get("coaching")

    return {
        "priority_metrics": {
            "pause_ratio": pause_info,
            "pitch_expressiveness": pitch_info,
            "loudness_variance": loudness_var_info,
            "crest_factor": crest_info,
            "speech_rate_wpm": wpm_info,
            "acoustic_speech_rate": voiced_info,
        },
        "supportive_metrics": {
            "volume_stability": volume_info,
        },
        "technical_reference": {
            "loudness_lufs": lufs_info,
        },
        "coaching": {
            "priority_recommendations": priority_coaching,
            "supportive_recommendations": supportive_coaching,
            "technical_note": technical_note,
        }
    }

# ============================================================
# RUN EVALUATION
# ============================================================

evaluation = evaluate_speech_performance(
    pause_ratio=pause_ratio,
    pitch_mean=pitch_mean,
    pitch_std=pitch_std,
    pitch_range=pitch_range,
    pitch_rms_deviation=pitch_rms_deviation,
    loudness_variance=loudness_variance,
    crest_factor_db=crest_factor_db,
    rms_mean=rms_mean,
    rms_std=rms_std,
    rms_cv=rms_cv,
    lufs=lufs,
    wpm=wpm,
    voiced_rate=voiced_rate,
    speaker_profile=None  # Set to "male" or "female" if known
)

print("\n" + "="*60)
print("SPEECH PERFORMANCE EVALUATION & COACHING")
print("="*60)

print("\n PRIORITY CAPTIVATION METRICS:")
for metric_name, metric_data in evaluation["priority_metrics"].items():
    print(f"\n{metric_name.upper().replace('_', ' ')}:")
    if "values" in metric_data:
        print(f"  Values: {metric_data['values']}")
        print(f"  Labels: {metric_data['labels']}")
    else:
        print(f"  Value: {metric_data.get('value')}")
        print(f"  Label: {metric_data.get('label')}")
    print(f"  📊 {metric_data.get('interpretation')}")

print("\n\n💡 TOP COACHING RECOMMENDATIONS:")
for i, rec in enumerate(evaluation["coaching"]["priority_recommendations"], 1):
    print(f"{i}. {rec}")

print("\n\n📋 SUPPORTIVE OBSERVATIONS:")
for rec in evaluation["coaching"]["supportive_recommendations"]:
    print(f"  • {rec}")

print("\n\n🔧 TECHNICAL NOTE:")
print(f"  {evaluation['coaching']['technical_note']}")
print(f"  {evaluation['technical_reference']['loudness_lufs']['note']}")


/Users/libiv/code/VERA/VERA-WhisperAudio/lib/python3.10/site-packages/faster_whisper/feature_extractor.py:224: RuntimeWarning: divide by zero encountered in matmul
  mel_spec = self.mel_filters @ magnitudes
/Users/libiv/code/VERA/VERA-WhisperAudio/lib/python3.10/site-packages/faster_whisper/feature_extractor.py:224: RuntimeWarning: overflow encountered in matmul
  mel_spec = self.mel_filters @ magnitudes
/Users/libiv/code/VERA/VERA-WhisperAudio/lib/python3.10/site-packages/faster_whisper/feature_extractor.py:224: RuntimeWarning: invalid value encountered in matmul
  mel_spec = self.mel_filters @ magnitudes



=== PRIORITY CAPTIVATION METRICS ===
Pause Ratio: 0.082 (fraction of non-speech time)
Pitch Std: 18.77 Hz (expressiveness)
Pitch Range: 114.85 Hz (vocal dynamism)
Pitch RMS Deviation: 18.77 Hz (prosodic melody)
Loudness Variance: 0.000748 (vocal emotionality)
Crest Factor: 19.53 dB (vocal punch/emphasis)
Speech Rate (WPM): 152.0
Voiced Frames/sec: 24.33 (fluency proxy)

=== SUPPORTIVE METRICS ===
Pitch Mean: 120.96 Hz (reference baseline)
RMS Mean/Std/CV: 0.0341 / 0.0274 / 0.802

=== NORMALIZATION REFERENCE ONLY ===
Integrated Loudness (LUFS): -27.1 (not for scoring engagement)

SPEECH PERFORMANCE EVALUATION & COACHING

 PRIORITY CAPTIVATION METRICS:

PAUSE RATIO:
  Value: 0.08200000000000003
  Label: low_pauses
  📊 Some pauses, but still quite continuous and fast-paced.

PITCH EXPRESSIVENESS:
  Values: {'mean_hz': 120.9617189297529, 'std_hz': 18.766169030162757, 'range_hz': 114.84884857877454, 'rms_deviation_hz': 18.766169030162757}
  Labels: {'std': 'low_variation', 'range': 'normal

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------
# {version 02-with comments}
# Feature extraction, Calculation, Labeling, Interpetation & Coaching
# ------------------------------------------------------------------------
# ------------------------------------------------------------------------

In [11]:
# ============================================================
# FEATURE EXTRACTION (Enhanced with Priority Metrics)
# ============================================================

# Path to the audio file that will be analyzed
AUDIO_PATH = "/Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_full_20251203_182015.mp3"

# 1) Load audio (mono, 16 kHz)

# Read the audio file. 'y' is the signal, 'sr' is the sample rate.
y, sr = sf.read(AUDIO_PATH)

# If the audio has more than one channel (stereo), convert it to mono
# by averaging all channels into a single one.
if y.ndim > 1:
    y = np.mean(y, axis=1)

# Target sample rate we want to work with (16 kHz is common for speech models).
target_sr = 16000

# If the original sample rate is not 16 kHz, resample it so everything
# is consistent for later processing.
if sr != target_sr:
    y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
    sr = target_sr

# Total duration of the audio in seconds.
duration_sec = len(y) / sr

# 2) VAD-based pause ratio with webrtcvad

# Create a Voice Activity Detector (VAD) object.
# The number 2 is the "aggressiveness" level: 0 is relaxed, 3 is very strict.
vad = webrtcvad.Vad(2)

# We will analyze the audio in small chunks (frames) of 30 milliseconds.
# The webrtcvad library only supports frame sizes of 10, 20, or 30 ms.
frame_ms = 30

# Convert 30 ms into a number of samples, given the sample rate.
frame_len = int(sr * frame_ms / 1000)

# How many complete frames of that length fit into the whole signal.
num_frames = len(y) // frame_len

# Counter for how many frames contain speech (according to VAD).
speech_frames = 0

# webrtcvad expects raw audio in 16-bit PCM format (integer samples as bytes),
# so we convert from float [-1, 1] to 16-bit integers and then to bytes.
pcm = (y * 32767).astype(np.int16).tobytes()

# Loop through the audio, one frame at a time.
for i in range(num_frames):
    # Each sample is 2 bytes (16-bit), so we multiply by 2 to get byte offsets.
    start = i * frame_len * 2
    end = start + frame_len * 2

    # Extract this small audio chunk from the byte string.
    frame = pcm[start:end]

    # If the last frame is shorter than expected, stop the loop.
    if len(frame) < frame_len * 2:
        break

    # Ask the VAD if this frame contains speech.
    if vad.is_speech(frame, sr):
        speech_frames += 1

# Convert number of speech frames into total speech time in seconds.
speech_time = speech_frames * frame_ms / 1000.0

# Pause time is total duration minus speech time.
pause_time = duration_sec - speech_time

# Pause ratio = fraction of time where there is no speech.
# The max(...) calls avoid division by zero and negative values.
pause_ratio = max(pause_time, 0.0) / max(duration_sec, 1e-6)

# 3) Pitch variation via librosa.pyin (F0 in Hz)

# Estimate the fundamental frequency (F0, perceived as pitch) across time.
# voiced_flag tells us which frames contain voiced sounds (where pitch is valid).
f0, voiced_flag, _ = librosa.pyin(
    y,
    fmin=librosa.note_to_hz("C2"),  # Minimum pitch to consider
    fmax=librosa.note_to_hz("C7"),  # Maximum pitch to consider
    sr=sr
)

# Remove frames where pitch could not be estimated (NaN values).
f0_voiced = f0[~np.isnan(f0)]

if len(f0_voiced) > 0:
    # Average pitch over voiced parts (baseline pitch height).
    pitch_mean = float(np.mean(f0_voiced))

    # Standard deviation of pitch: how much pitch moves around on average.
    pitch_std = float(np.std(f0_voiced))

    # Pitch range: difference between the highest and lowest pitch.
    pitch_range = float(np.max(f0_voiced) - np.min(f0_voiced))

    # Pitch RMS Deviation: another way to measure how much pitch varies
    # around the mean. It is similar to standard deviation, but we
    # compute it explicitly to make the idea clearer.
    pitch_rms_deviation = float(np.sqrt(np.mean((f0_voiced - pitch_mean) ** 2)))
else:
    # If no voiced frames were found, set pitch-related values to NaN.
    pitch_mean = pitch_std = pitch_range = pitch_rms_deviation = np.nan

# 4) Volume stability via RMS

# For loudness over time, we use short frames (50 ms) and overlap them by half.
frame_len_rms = int(0.05 * sr)  # 50 ms in samples
hop_len_rms = frame_len_rms // 2

# Compute RMS (Root Mean Square) energy over time.
# This approximates how loud the signal is in each short segment.
rms = librosa.feature.rms(y=y, frame_length=frame_len_rms, hop_length=hop_len_rms)[0]

# Average loudness across the whole signal.
rms_mean = float(np.mean(rms))

# How much loudness changes around the mean.
rms_std = float(np.std(rms))

# Coefficient of variation (std / mean): relative variation in loudness.
rms_cv = float(rms_std / (rms_mean + 1e-8))

# Short-term Loudness Variance (direct emotionality indicator)
# This is the variance of the RMS values across frames and shows
# how much the loudness moves up and down over time.
loudness_variance = float(np.var(rms))

# Crest Factor (dynamic range / vocal punch)
# Crest Factor in dB = 20 * log10(peak amplitude / RMS amplitude).
# Higher values usually mean stronger emphasis and more dynamic speech.
peak_amplitude = float(np.max(np.abs(y)))
overall_rms = float(np.sqrt(np.mean(y ** 2)))
if overall_rms > 1e-8:
    crest_factor_db = float(20 * np.log10(peak_amplitude / overall_rms))
else:
    # If the signal is almost silent, avoid division by zero.
    crest_factor_db = 0.0

# 5) Integrated loudness (LUFS) - for normalization reference only

# LUFS is a standard way to measure perceived loudness of audio.
# It is used here only as a technical reference, not to judge speaking style.
meter = pyln.Meter(sr)
lufs = float(meter.integrated_loudness(y))

# 6) Speech rate via faster-whisper (WPM)

# Load the Whisper model to get an automatic transcription.
# "small" is the model size, and "cpu"/"int8" keep it light for most machines.
model = WhisperModel("small", device="cpu", compute_type="int8")

# Get transcription segments, each with text and time stamps.
segments, _ = model.transcribe(AUDIO_PATH, beam_size=1)

words = 0
first_t = None
last_t = None

# Loop over all segments to count words and track speaking time.
for seg in segments:
    text = seg.text.strip()
    if not text:
        continue

    # Count how many words are in this segment.
    seg_words = text.split()
    words += len(seg_words)

    # Mark the time of the first and last spoken segments.
    if first_t is None:
        first_t = seg.start
    last_t = seg.end

# If there is any speech, compute Words Per Minute (WPM).
if words > 0 and last_t is not None and first_t is not None:
    spoken_dur_min = (last_t - first_t) / 60.0
    # WPM = total words divided by speaking duration in minutes.
    wpm = words / max(spoken_dur_min, 1e-6)
else:
    # If we did not detect any words, speech rate is zero.
    wpm = 0.0

# 7) Acoustic speech rate proxy: voiced frames per second from pyin

# voiced_flag is a boolean array saying which frames are voiced.
# We estimate how often voice is present per second.
# This gives a rough idea of how continuously someone is speaking.
if duration_sec > 0:
    voiced_rate = float(np.mean(voiced_flag)) * (len(voiced_flag) / duration_sec)
else:
    voiced_rate = 0.0

# Print the main metrics that directly relate to how captivating the speech is.
print("\n=== PRIORITY CAPTIVATION METRICS ===")
print(f"Pause Ratio: {pause_ratio:.3f} (fraction of non-speech time)")
print(f"Pitch Std: {pitch_std:.2f} Hz (expressiveness)")
print(f"Pitch Range: {pitch_range:.2f} Hz (vocal dynamism)")
print(f"Pitch RMS Deviation: {pitch_rms_deviation:.2f} Hz (prosodic melody)")
print(f"Loudness Variance: {loudness_variance:.6f} (vocal emotionality)")
print(f"Crest Factor: {crest_factor_db:.2f} dB (vocal punch/emphasis)")
print(f"Speech Rate (WPM): {wpm:.1f}")
print(f"Voiced Frames/sec: {voiced_rate:.2f} (fluency proxy)")

# Print helpful but secondary metrics about volume behaviour.
print("\n=== SUPPORTIVE METRICS ===")
print(f"Pitch Mean: {pitch_mean:.2f} Hz (reference baseline)")
print(f"RMS Mean/Std/CV: {rms_mean:.4f} / {rms_std:.4f} / {rms_cv:.3f}")

# Print technical loudness information used only for normalization,
# not for judging how engaging the speaker is.
print("\n=== NORMALIZATION REFERENCE ONLY ===")
print(f"Integrated Loudness (LUFS): {lufs:.1f} (not for scoring engagement)")

# ============================================================
# INTERPRETATION FUNCTIONS (Updated Priority)
# ============================================================

def interpret_pause_ratio(pause_ratio: float) -> Dict[str, Any]:
    """
    Interpret pause ratio - HIGH PRIORITY for captivation.

    Takes the fraction of silence in the audio and returns:
    - A label (text category)
    - A human-friendly explanation
    - A simple coaching suggestion
    """
    if pause_ratio < 0.05:
        label = "very_low_pauses"
        interpretation = "Almost no silence; speech likely feels rushed or dense."
        coaching = "Add short pauses (0.5-1 second) after key ideas for comprehension."
    elif pause_ratio < 0.15:
        label = "low_pauses"
        interpretation = "Some pauses, but still quite continuous and fast-paced."
        coaching = "Use slightly longer pauses after important points for emphasis."
    elif pause_ratio < 0.35:
        label = "balanced_pauses"
        interpretation = "Healthy balance; rhythm likely feels natural and engaging."
        coaching = "Excellent pause usage; maintain this natural rhythm."
    elif pause_ratio < 0.50:
        label = "many_pauses"
        interpretation = "Many or long pauses; aids clarity but may sound hesitant."
        coaching = "Connect sentences more smoothly while keeping key strategic pauses."
    else:
        label = "very_high_pauses"
        interpretation = "Over 50% silence; speech may feel fragmented or disjointed."
        coaching = "Reduce long pauses and maintain more continuous flow between ideas."

    return {
        "value": pause_ratio,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_pitch_expressiveness(
    pitch_mean: float,
    pitch_std: float,
    pitch_range: float,
    pitch_rms_deviation: float,
    speaker_profile: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Interpret pitch-based expressiveness - HIGH PRIORITY for captivation.

    Uses:
    - pitch_std: how much pitch moves overall
    - pitch_range: lowest to highest pitch span
    - pitch_rms_deviation: smooth measure of how melodic the voice is

    Returns labels, explanations, and coaching tips.
    """

    # Pitch Std (High Priority) - how "flat" or "lively" the pitch is
    if pitch_std < 15:
        std_label = "very_flat"
        std_interp = "Minimal pitch movement; sounds monotone or robotic."
        std_coaching = "Add pitch rises on important words and falls at sentence ends."
    elif pitch_std < 25:
        std_label = "low_variation"
        std_interp = "Limited pitch movement; functional but not expressive."
        std_coaching = "Increase pitch contrast on key ideas to sound more dynamic."
    elif pitch_std < 40:
        std_label = "moderate"
        std_interp = "Some pitch movement; approaching engaging levels."
        std_coaching = "Good baseline; push slightly more variation on emotional content."
    elif pitch_std < 60:
        std_label = "expressive"
        std_interp = "Strong pitch movement; natural and highly engaging."
        std_coaching = "Excellent expressiveness; maintain this dynamic quality."
    else:
        std_label = "very_wide"
        std_interp = "Very large pitch movement; theatrical and highly emotional."
        std_coaching = "Consider smoothing extremes unless intentionally dramatic."

    # Pitch Range (High Priority) - total span of pitch used
    if pitch_range < 50:
        range_label = "very_narrow"
        range_interp = "Extremely limited pitch span; reinforces monotone."
        range_coaching = "Practice using wider pitch on questions and emphasis words."
    elif pitch_range < 100:
        range_label = "narrow"
        range_interp = "Limited pitch span; reduces vocal dynamism."
        range_coaching = "Expand your pitch range on key phrases for more energy."
    elif pitch_range < 200:
        range_label = "normal"
        range_interp = "Typical pitch span for conversational speech."
        range_coaching = "Good range; focus on strategic placement of pitch changes."
    elif pitch_range < 300:
        range_label = "wide"
        range_interp = "Large pitch span; indicates expressive delivery."
        range_coaching = "Excellent vocal dynamism; ensure it matches your message."
    else:
        range_label = "very_wide"
        range_interp = "Very large pitch span; highly theatrical or emotional."
        range_coaching = "Maintain this range only if appropriate for your audience and context."

    # Pitch RMS Deviation (High Priority - "vocal melody")
    if pitch_rms_deviation < 15:
        rms_label = "flat_prosody"
        rms_interp = "Low melodic variation; lacks charismatic vocal melody."
        rms_coaching = "Work on creating more 'musical' pitch patterns in your speech."
    elif pitch_rms_deviation < 25:
        rms_label = "moderate_prosody"
        rms_interp = "Some melodic quality; could be more captivating."
        rms_coaching = "Enhance prosodic melody by varying pitch more on important words."
    elif pitch_rms_deviation < 45:
        rms_label = "melodic"
        rms_interp = "Good vocal melody; engaging prosody that holds attention."
        rms_coaching = "Strong prosodic expressiveness; this enhances your charisma."
    else:
        rms_label = "highly_melodic"
        rms_interp = "Very high vocal melody; highly captivating prosody."
        rms_coaching = "Exceptional melodic expressiveness; maintain for high-impact moments."

    # Mean context (reference only, not for scoring)
    # This uses typical male/female pitch ranges just as a reference,
    # not to say what is "good" or "bad".
    mean_context = None
    if speaker_profile == "male":
        if pitch_mean < 80:
            mean_context = "Mean pitch very low for typical adult male."
        elif pitch_mean > 155:
            mean_context = "Mean pitch relatively high for typical adult male."
        else:
            mean_context = "Mean pitch within typical adult male range."
    elif speaker_profile == "female":
        if pitch_mean < 165:
            mean_context = "Mean pitch relatively low for typical adult female."
        elif pitch_mean > 255:
            mean_context = "Mean pitch relatively high for typical adult female."
        else:
            mean_context = "Mean pitch within typical adult female range."

    # Combine coaching messages into one compact string.
    coaching = f"Variation: {std_coaching} Range: {range_coaching} Prosody: {rms_coaching}"

    return {
        "values": {
            "mean_hz": pitch_mean,
            "std_hz": pitch_std,
            "range_hz": pitch_range,
            "rms_deviation_hz": pitch_rms_deviation,
        },
        "labels": {
            "std": std_label,
            "range": range_label,
            "rms_deviation": rms_label,
        },
        "interpretation": {
            "std": std_interp,
            "range": range_interp,
            "rms_deviation": rms_interp,
            "mean_context": mean_context,
        },
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_loudness_variance(loudness_variance: float) -> Dict[str, Any]:
    """
    Interpret short-term loudness variance - HIGH PRIORITY for emotionality.

    Higher variance usually means the speaker uses louder and softer parts
    to express emotion. Very low variance can sound flat.
    """
    # These thresholds are rough and may need tuning for your dataset.
    if loudness_variance < 0.0001:
        label = "very_flat"
        interpretation = "Extremely consistent loudness; lacks emotional texture."
        coaching = "Add dynamic loudness changes to emphasize important moments."
    elif loudness_variance < 0.0005:
        label = "low_variation"
        interpretation = "Low loudness variation; sounds controlled but flat."
        coaching = "Increase volume on key words and soften transitions for more emotion."
    elif loudness_variance < 0.002:
        label = "moderate"
        interpretation = "Moderate loudness variation; natural vocal dynamics."
        coaching = "Good emotional texture; use louder emphasis strategically."
    elif loudness_variance < 0.005:
        label = "high_variation"
        interpretation = "High loudness variation; emotionally expressive delivery."
        coaching = "Excellent dynamic range; ensure quieter parts remain audible."
    else:
        label = "very_high"
        interpretation = "Very high loudness swings; highly emotional or inconsistent."
        coaching = "Maintain steadier baseline while preserving intentional emphasis."

    return {
        "value": loudness_variance,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_crest_factor(crest_factor_db: float) -> Dict[str, Any]:
    """
    Interpret crest factor - HIGH PRIORITY for vocal punch/excitement.

    Crest factor compares peak loudness to average loudness.
    Typical speech is around 12 dB; much lower can sound flat,
    much higher can sound very punchy or uneven.
    """
    if crest_factor_db < 8:
        label = "very_compressed"
        interpretation = "Very low dynamic range; sounds flat and overly compressed."
        coaching = "Add more articulation emphasis with sharper consonants and stressed syllables."
    elif crest_factor_db < 11:
        label = "low_dynamics"
        interpretation = "Below typical speech dynamics; lacks punch and excitement."
        coaching = "Increase contrast between emphasized and normal words for more impact."
    elif crest_factor_db < 14:
        label = "normal"
        interpretation = "Typical speech dynamics; natural balance of peaks and average level."
        coaching = "Good dynamic balance; maintain clear articulation on key points."
    elif crest_factor_db < 18:
        label = "high_dynamics"
        interpretation = "High dynamic range; energetic with strong articulation bursts."
        coaching = "Excellent vocal punch; ensure average level remains comfortable."
    else:
        label = "very_high"
        interpretation = "Very high dynamic range; extremely punchy or inconsistent."
        coaching = "Consider smoothing extreme peaks while maintaining overall energy."

    return {
        "value": crest_factor_db,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_speech_rate_wpm(wpm: float) -> Dict[str, Any]:
    """
    Interpret speech rate - HIGH PRIORITY for engagement.

    Uses words per minute (WPM) to judge if the pace is too slow,
    too fast, or in a comfortable range for most listeners.
    """
    if wpm < 100:
        label = "very_slow"
        interpretation = "Very slow pace; can feel heavy or overly deliberate."
        coaching = "Increase tempo on less critical details to add energy."
    elif wpm < 130:
        label = "slow"
        interpretation = "Slower than typical; very clear but may lack energy."
        coaching = "Speed up slightly while maintaining clarity on important points."
    elif wpm < 160:
        label = "optimal"
        interpretation = "Optimal range for presentations; easy to follow and engaging."
        coaching = "Excellent pace; maintain this speed and use pauses for emphasis."
    elif wpm < 190:
        label = "fast"
        interpretation = "Fast pace; energetic but can reduce comprehension."
        coaching = "Slow down slightly and add clearer pauses between key points."
    else:
        label = "very_fast"
        interpretation = "Very fast; listeners may miss information or feel overwhelmed."
        coaching = "Deliberately slow down and pause more so important points land."

    return {
        "value": wpm,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_acoustic_speech_rate(voiced_rate: float) -> Dict[str, Any]:
    """
    Interpret acoustic speech rate proxy - HIGH PRIORITY for fluency.

    voiced_rate is an estimate of how often the voice is active per second.
    Higher values mean more continuous speech and fewer pauses.
    """
    if voiced_rate < 15:
        label = "sparse_voicing"
        interpretation = "Very low voicing density; suggests slow speech or many pauses."
        coaching = "Reduce pause lengths slightly or increase speaking tempo."
    elif voiced_rate < 22:
        label = "below_average"
        interpretation = "Lower voicing density; deliberate pacing or pause-heavy."
        coaching = "Ensure pauses are strategic, not hesitation-based."
    elif voiced_rate < 30:
        label = "normal"
        interpretation = "Typical voicing density for moderate-paced speech."
        coaching = "Good fluency baseline; combine with clear pauses and articulation."
    elif voiced_rate < 38:
        label = "high_density"
        interpretation = "Dense voicing; suggests fast speech or minimal pauses."
        coaching = "Insert brief pauses after key ideas to avoid sounding rushed."
    else:
        label = "very_high"
        interpretation = "Very dense voicing; rapid speech with very little silence."
        coaching = "Add breathing room with short pauses so listeners can process."

    return {
        "value": voiced_rate,
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "HIGH"
    }

def interpret_volume_stability(rms_mean: float, rms_std: float, rms_cv: float) -> Dict[str, Any]:
    """
    Interpret volume stability - SUPPORTIVE metric (not primary).

    Looks at how stable or variable loudness is during the speech.
    Very high variation can be distracting; very low can be too flat.
    """
    if rms_cv < 0.30:
        label = "very_stable"
        interpretation = "Very consistent loudness; highly controlled."
        coaching = "If too flat, add small dynamic emphasis on key sentences."
    elif rms_cv < 0.60:
        label = "stable"
        interpretation = "Moderate volume changes; sounds natural."
        coaching = "Good stability; ensure quieter words remain audible."
    elif rms_cv < 1.00:
        label = "variable"
        interpretation = "Noticeable volume swings; some inconsistency."
        coaching = "Maintain steadier microphone distance and avoid trailing off."
    else:
        label = "highly_variable"
        interpretation = "Strong volume swings; can be distracting."
        coaching = "Aim for more consistent baseline; reserve big changes for emphasis."

    return {
        "values": {
            "rms_mean": rms_mean,
            "rms_std": rms_std,
            "rms_cv": rms_cv,
        },
        "label": label,
        "interpretation": interpretation,
        "coaching": coaching,
        "priority": "SUPPORTIVE"
    }

def interpret_loudness_lufs(lufs: float) -> Dict[str, Any]:
    """
    Interpret LUFS - NORMALIZATION REFERENCE ONLY (not for scoring engagement).

    LUFS mainly reflects recording level (gain, mic distance),
    not speaking skill. It is used as a technical guide only.
    """
    if lufs < -26:
        label = "very_quiet"
        note = "Recording level is low; this is a technical issue, not speech quality."
        coaching = "Increase recording gain or move closer to microphone (aim for -20 to -18 LUFS)."
    elif lufs < -22:
        label = "quiet"
        note = "Recording below comfortable range; technical adjustment needed."
        coaching = "Raise recording level slightly for better playback comfort."
    elif lufs < -18:
        label = "comfortable"
        note = "Recording level is in good range; no technical adjustments needed."
        coaching = "Loudness normalization is appropriate; focus on performance metrics."
    elif lufs < -14:
        label = "loud"
        note = "Recording louder than typical speech; may cause listener fatigue."
        coaching = "Lower recording level slightly to reduce fatigue risk."
    else:
        label = "very_loud"
        note = "Recording is very loud; risk of distortion and listener fatigue."
        coaching = "Reduce recording gain or move further from microphone."

    return {
        "value": lufs,
        "label": label,
        "interpretation": note,
        "coaching": coaching,
        "priority": "NORMALIZATION_ONLY",
        "note": "Use only for technical normalization, not for scoring engagement."
    }

# ============================================================
# COMPREHENSIVE SPEECH EVALUATION
# ============================================================

def evaluate_speech_performance(
    pause_ratio: float,
    pitch_mean: float,
    pitch_std: float,
    pitch_range: float,
    pitch_rms_deviation: float,
    loudness_variance: float,
    crest_factor_db: float,
    rms_mean: float,
    rms_std: float,
    rms_cv: float,
    lufs: float,
    wpm: float,
    voiced_rate: float,
    speaker_profile: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Comprehensive speech evaluation focused on captivation metrics.

    Uses all previously computed features to:
    - Classify the speaking style
    - Provide human-readable interpretations
    - Offer coaching recommendations
    """

    # Interpret all high-priority features.
    pause_info = interpret_pause_ratio(pause_ratio)
    pitch_info = interpret_pitch_expressiveness(
        pitch_mean, pitch_std, pitch_range, pitch_rms_deviation, speaker_profile
    )
    loudness_var_info = interpret_loudness_variance(loudness_variance)
    crest_info = interpret_crest_factor(crest_factor_db)
    wpm_info = interpret_speech_rate_wpm(wpm)
    voiced_info = interpret_acoustic_speech_rate(voiced_rate)

    # Supportive metrics (extra info that adds context).
    volume_info = interpret_volume_stability(rms_mean, rms_std, rms_cv)
    lufs_info = interpret_loudness_lufs(lufs)

    # Collect coaching from all high-priority metrics in a single list.
    priority_coaching = []
    for item in [pause_info, pitch_info, loudness_var_info, crest_info, wpm_info, voiced_info]:
        if item.get("priority") == "HIGH":
            priority_coaching.append(item.get("coaching"))

    # Supportive coaching from volume stability.
    supportive_coaching = [volume_info.get("coaching")]

    # Technical note from loudness (not performance coaching).
    technical_note = lufs_info.get("coaching")

    return {
        "priority_metrics": {
            "pause_ratio": pause_info,
            "pitch_expressiveness": pitch_info,
            "loudness_variance": loudness_var_info,
            "crest_factor": crest_info,
            "speech_rate_wpm": wpm_info,
            "acoustic_speech_rate": voiced_info,
        },
        "supportive_metrics": {
            "volume_stability": volume_info,
        },
        "technical_reference": {
            "loudness_lufs": lufs_info,
        },
        "coaching": {
            "priority_recommendations": priority_coaching,
            "supportive_recommendations": supportive_coaching,
            "technical_note": technical_note,
        }
    }

# ============================================================
# RUN EVALUATION
# ============================================================

# Run the full evaluation using all the metrics we computed above.
evaluation = evaluate_speech_performance(
    pause_ratio=pause_ratio,
    pitch_mean=pitch_mean,
    pitch_std=pitch_std,
    pitch_range=pitch_range,
    pitch_rms_deviation=pitch_rms_deviation,
    loudness_variance=loudness_variance,
    crest_factor_db=crest_factor_db,
    rms_mean=rms_mean,
    rms_std=rms_std,
    rms_cv=rms_cv,
    lufs=lufs,
    wpm=wpm,
    voiced_rate=voiced_rate,
    speaker_profile=None  # Set to "male" or "female" if known
)

print("\n" + "="*60)
print("SPEECH PERFORMANCE EVALUATION AND COACHING")
print("="*60)

# Show each priority metric, its values/labels, and the interpretation text.
print("\nPRIORITY CAPTIVATION METRICS:")
for metric_name, metric_data in evaluation["priority_metrics"].items():
    print(f"\n{metric_name.upper().replace('_', ' ')}:")
    if "values" in metric_data:
        print(f"  Values: {metric_data['values']}")
        print(f"  Labels: {metric_data['labels']}")
    else:
        print(f"  Value: {metric_data.get('value')}")
        print(f"  Label: {metric_data.get('label')}")
    print(f"  Interpretation: {metric_data.get('interpretation')}")

# Print top coaching recommendations, ordered in a simple list.
print("\nTOP COACHING RECOMMENDATIONS:")
for i, rec in enumerate(evaluation["coaching"]["priority_recommendations"], 1):
    print(f"{i}. {rec}")

# Print supportive (secondary) observations.
print("\nSUPPORTIVE OBSERVATIONS:")
for rec in evaluation["coaching"]["supportive_recommendations"]:
    print(f"  - {rec}")

# Print technical notes related to recording loudness.
print("\nTECHNICAL NOTE:")
print(f"  {evaluation['coaching']['technical_note']}")
print(f"  {evaluation['technical_reference']['loudness_lufs']['note']}")


/Users/libiv/code/VERA/VERA-WhisperAudio/lib/python3.10/site-packages/faster_whisper/feature_extractor.py:224: RuntimeWarning: divide by zero encountered in matmul
  mel_spec = self.mel_filters @ magnitudes
/Users/libiv/code/VERA/VERA-WhisperAudio/lib/python3.10/site-packages/faster_whisper/feature_extractor.py:224: RuntimeWarning: overflow encountered in matmul
  mel_spec = self.mel_filters @ magnitudes
/Users/libiv/code/VERA/VERA-WhisperAudio/lib/python3.10/site-packages/faster_whisper/feature_extractor.py:224: RuntimeWarning: invalid value encountered in matmul
  mel_spec = self.mel_filters @ magnitudes



=== PRIORITY CAPTIVATION METRICS ===
Pause Ratio: 0.081 (fraction of non-speech time)
Pitch Std: 18.78 Hz (expressiveness)
Pitch Range: 114.85 Hz (vocal dynamism)
Pitch RMS Deviation: 18.78 Hz (prosodic melody)
Loudness Variance: 0.000745 (vocal emotionality)
Crest Factor: 19.63 dB (vocal punch/emphasis)
Speech Rate (WPM): 149.6
Voiced Frames/sec: 23.80 (fluency proxy)

=== SUPPORTIVE METRICS ===
Pitch Mean: 120.90 Hz (reference baseline)
RMS Mean/Std/CV: 0.0335 / 0.0273 / 0.814

=== NORMALIZATION REFERENCE ONLY ===
Integrated Loudness (LUFS): -27.2 (not for scoring engagement)

SPEECH PERFORMANCE EVALUATION AND COACHING

PRIORITY CAPTIVATION METRICS:

PAUSE RATIO:
  Value: 0.0812743823146944
  Label: low_pauses
  Interpretation: Some pauses, but still quite continuous and fast-paced.

PITCH EXPRESSIVENESS:
  Values: {'mean_hz': 120.89590343020599, 'std_hz': 18.782650308346742, 'range_hz': 114.84884857877454, 'rms_deviation_hz': 18.782650308346742}
  Labels: {'std': 'low_variation', '